# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.22,81,4,8.38,SH,1699816378
1,1,kodiak,57.7900,-152.4072,1.73,60,0,2.06,US,1699816349
2,2,as sulayyil,20.4607,45.5779,23.14,37,77,3.79,SA,1699816379
3,3,stanley,54.8680,-1.6985,6.51,92,75,3.60,GB,1699816379
4,4,vilyuysk,63.7553,121.6247,-20.24,99,42,2.16,RU,1699816379


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", size = "Humidity", color = "City"    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Humidity'].astype(int)<30),:]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,sarmiento,-45.5882,-69.0700,15.68,19,6,8.54,AR,1699816397
58,58,upington,-28.4478,21.2561,24.21,19,0,2.57,ZA,1699816399
109,109,inyati,-19.6756,28.8469,24.18,17,14,3.34,ZW,1699816419
122,122,sao vicente,-23.9631,-46.3919,41.26,26,97,0.89,BR,1699816424
144,144,esmeraldas,-19.7625,-44.3139,37.08,18,10,1.86,BR,1699816430


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
del hotel_df['City_ID']
del hotel_df['Max Temp']
del hotel_df['Cloudiness']
del hotel_df['Wind Speed']
del hotel_df['Country']
del hotel_df['Date']
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Hotel Name
54,sarmiento,-45.5882,-69.0700,19,
58,upington,-28.4478,21.2561,19,
109,inyati,-19.6756,28.8469,17,
122,sao vicente,-23.9631,-46.3919,26,
144,esmeraldas,-19.7625,-44.3139,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = { "categories": categories, "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sarmiento - nearest hotel: Ismar
upington - nearest hotel: Die Eiland Holiday Ressort
inyati - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
esmeraldas - nearest hotel: No hotel found
coronel suarez - nearest hotel: La Casa Hotel Boutique
kayes - nearest hotel: No hotel found
villamontes - nearest hotel: Hoterma
kutum - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
sidi khaled - nearest hotel: فندق ترنزيت
alto araguaia - nearest hotel: Hotel Araguaia
ivins - nearest hotel: Crescent Moon Inn
hit - nearest hotel: No hotel found
afif - nearest hotel: Samaya Royal Afif
el bauga - nearest hotel: No hotel found
johi - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
sume - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
masjed soleyman - nearest hotel: هتل جهانگردی
robore - nearest hotel: Lajas
tanout - nearest hotel: No hotel found
high river - nearest hotel: High River Inn

,City,Lat,Lng,Humidity,Hotel Name
54,sarmiento,-45.5882,-69.0700,19,Ismar
58,upington,-28.4478,21.2561,19,Die Eiland Holiday Ressort
109,inyati,-19.6756,28.8469,17,No hotel found
122,sao vicente,-23.9631,-46.3919,26,Pousada Vitória
144,esmeraldas,-19.7625,-44.3139,18,No hotel found
156,coronel suarez,-37.4547,-61.9334,19,La Casa Hotel Boutique
158,kayes,14.0000,-11.0000,14,No hotel found
162,villamontes,-21.2500,-63.5000,18,Hoterma
192,kutum,14.2000,24.6667,27,No hotel found
197,bardai,11.6500,7.2333,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_df['Country'] = ideal_weather_df['Country']

map_plot_2 = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo = True,
    tiles = "OSM", color = "City", hover_cols='all')

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,index,Hotel Name,Country)